In [ ]:
import numpy as np
import pathlib
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.signal as signal
import h5py


In [ ]:
path2data = pathlib.Path.home()/'group'/'project'/'results'/'beamforming'/\
'time_dependent'/'beamform_signals'/'220505_dl_grid_84to90deg_1to45mm.h5'

path2save = pathlib.Path.home()/'group'/'project'/'results'/'beamforming'/\
'time_dependent'/'beamform_signals'/'220506_dl_grid_87.5to90deg_1to3mm_fftind.npy'

#data = np.load(path2data)['x']
file = h5py.File(path2data, 'r')
#data = file['x'][:, 0:8192]

signal_energy = file['energy'][:]
signal_pitch = file['pitch'][:]
signal_radius = file['radius'][:]

select_energy = np.linspace(18545, 18555, 11)
select_pitch = np.linspace(87.5, 90, 51)
select_radius = np.array([0.001, 0.002, 0.003])

energy_grid, pitch_grid, radius_grid = np.meshgrid(
            select_energy,
            select_pitch,
            select_radius
             )

inds = []
for i, pair in enumerate(zip(energy_grid.flatten(), pitch_grid.flatten(), radius_grid.flatten())):

    #print(pair)
    try:
        index = np.argwhere(
            np.logical_and(abs(pair[0] - signal_energy) < 1e-5, 
            np.logical_and(abs(pair[1] - signal_pitch) < 1e-5, abs(pair[2] - signal_radius) < 1e-5))
        ).squeeze()
        if index.size == 1:
            inds.append(index)
    except:
        pass

inds = np.sort(np.array(inds, dtype=np.int32))

data = file['x'][inds, 0:8192]


fft_data = np.fft.fftshift(np.fft.fft(data, axis=-1) / 8192)

peak_list = []
for n in range(data.shape[0]):
    ex_spectrum = abs(fft_data[n, :])
    peaks = signal.find_peaks(ex_spectrum ** 2, threshold = 0.06e-14)

    peak_list.extend(list(peaks[0]))
    #for i in range(peaks[0].size - 1):
    #    dist = peaks[0][i+1] - peaks[0][i]
    #    dist_list.append(dist)
        
peak_array = np.array(peak_list)

N, bins = np.histogram(
    peak_array, 
    bins=np.arange(1000, 8000, 1), 
    density=True
)

select_inds = np.argwhere(N >=0.000001).squeeze()
select_bins = bins[1:][select_inds]
print(select_bins.size)
select_data = fft_data[:, select_bins]

np.save(path2save, select_bins)


#path2data =pathlib.Path.home()/'group'/'project'/'datasets'/'data'/\
#'bf'/'220301_dense_grid_87.0deg_2cm.npy'

file.close()

In [ ]:
plt.plot(bins[1:], N)

In [ ]:
data = np.load(path2data)

In [ ]:
data.shape

In [ ]:
sns.set_theme(context='talk')
fig = plt.figure(figsize=(13, 8))
ax = fig.add_subplot(1,1,1)

ax.plot(data[0, :].real)
ax.plot(data[0, :].imag)

In [ ]:
fft_data = np.fft.fftshift(np.fft.fft(data, axis=-1) / 8192)

In [ ]:
ex_ind = 0
ex_spectrum = abs(fft_data[ex_ind, :])

In [ ]:
noise = np.random.multivariate_normal(
    [0, 0], np.eye(2) * 10 * 50 * 60 * np.sqrt(1) * 1.38e-23 * 200e6 / (2 * 8192), 8192
)
noise = noise[:, 0] + 1j*noise[:, 1]

In [ ]:
sns.set_theme(context='talk')
fig = plt.figure(figsize=(13, 8))
ax = fig.add_subplot(1,1,1)

ax.plot(abs(noise) ** 2)
ax.plot(ex_spectrum ** 2)

In [ ]:
from scipy import signal

In [ ]:
peaks = signal.find_peaks(ex_spectrum ** 2, threshold = 0.06e-14)
print(peaks)

In [ ]:
for i in range(peaks[0].size - 1):
    print(peaks[0][i+1] - peaks[0][i])

In [ ]:
dist_list = []
for n in range(data.shape[0]):
    ex_spectrum = abs(fft_data[n, :])
    peaks = signal.find_peaks(ex_spectrum ** 2, threshold = 0.06e-14)
    
    for i in range(peaks[0].size - 1):
        dist = peaks[0][i+1] - peaks[0][i]
        dist_list.append(dist)
        
dist_array = np.array(dist_list)
print(dist_array.mean(), dist_array.max(), dist_array.min())


In [ ]:
sns.set_theme(context='talk', style='whitegrid')
fig = plt.figure(figsize=(13, 8))
ax = fig.add_subplot(1,1,1)

hist = ax.hist(dist_array, 256)

# Downselect peak regions, then analyze peak separation

In [ ]:
peak_list = []
for n in range(data.shape[0]):
    ex_spectrum = abs(fft_data[n, :])
    peaks = signal.find_peaks(ex_spectrum ** 2, threshold = 0.06e-14)

    peak_list.extend(list(peaks[0]))
    #for i in range(peaks[0].size - 1):
    #    dist = peaks[0][i+1] - peaks[0][i]
    #    dist_list.append(dist)
        
peak_array = np.array(peak_list)
#print(dist_array.mean(), dist_array.max(), dist_array.min())

In [ ]:
sns.set_theme(context='talk', style='whitegrid')
fig = plt.figure(figsize=(13, 8))
ax = fig.add_subplot(1,1,1)

hist = ax.hist(peak_array, bins=200, density=True)

In [ ]:
N, bins = np.histogram(
    peak_array, 
    bins=np.arange(1000, 8000, 1), 
    density=True
)

In [ ]:
select_inds = np.argwhere(N >=0.000001).squeeze()
select_bins = bins[1:][select_inds]

In [ ]:
sns.set_theme(context='talk', style='whitegrid')
fig = plt.figure(figsize=(13, 8))
ax = fig.add_subplot(1,1,1)

ax.plot(bins[1:], N, )

ax.plot(bins[1:][select_inds], N[select_inds], 'x')

In [ ]:
print(select_inds[0:100], select_bins[0:100], select_bins.size)

In [ ]:
select_data = fft_data[:, select_bins]

In [ ]:
ex_ind = 0
ex_spectrum = abs(fft_data[ex_ind, :])
#ex_spectrum = abs(select_data[ex_ind, :])

sns.set_theme(context='talk', style='whitegrid')
fig = plt.figure(figsize=(13, 8))
ax = fig.add_subplot(1,1,1)

#ax.plot(ex_spectrum ** 2)
ax.plot(ex_spectrum[select_bins] ** 2)





In [ ]:
dist_list = []
for n in range(data.shape[0]):
    ex_spectrum = abs(select_data[n, :])
    peaks = signal.find_peaks(ex_spectrum ** 2, threshold = 0.5e-14)
    
    for i in range(peaks[0].size - 1):
        dist = peaks[0][i+1] - peaks[0][i]
        dist_list.append(dist)
        
dist_array = np.array(dist_list)
print(dist_array.mean(), dist_array.max(), dist_array.min())

In [ ]:
np.save(path2save, select_bins)